# Learn pipe operator

In [1]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            print("invoke")
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [6]:
class MyRunnable:
    def __init__(self, func):
        self.func = func
        self.mem = None

    def __setmem__(self, mem):
        print("Install mem")
        self.mem = mem
    
    def __or__(self, other):
        print("chained")
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            print("invoke")
            res = self.func(*args, **kwargs)
            return other(res)
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        res = self.func(*args, **kwargs)
        self.mem.append(res)
        return res

In [7]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
mem = []
add_five = Runnable(add_five)
add_two_store = MyRunnable(lambda e: e + 2)
add_two_store.__setmem__(mem)
multiply_by_two = Runnable(multiply_by_two)

etl = add_five | add_two_store | add_five
print(etl(3))
mem

Install mem
invoke
invoke
15


[10]

In [23]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
mem = []
add_five = Runnable(add_five, mem)
multiply_by_two = Runnable(multiply_by_two, mem)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16


chained
call
invoke
call


16

In [8]:
def sum(data):
    return data['a'] + data['b']

sum = Runnable(sum)

a = {"a": 1, "b":2} | sum
a

TypeError: unsupported operand type(s) for |: 'dict' and 'Runnable'